<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-11 16:23:56
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -1.01 L (-0.74%)
Current PnL: -14.00 L (-9.76%)
CY Booked + Current PnL: -7.04 L (-4.91%)
-------------------
Total profit:  4.68 L
Total loss:  -18.68 L
-------------------
Total Booked + Current PnL: 19.94 L (17.11%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.11%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.54 L (53.29%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-223.38,H-SC,94.79,89027.0,-6289.0,6330.0,1.73,-6.60,7.11,0.04,157.0,-0.99,0.65,75.60,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,54.17,214292.0,12812.0,14400.0,0.08,6.36,6.72,13.51,77.0,0.89,1.58,19.76,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-22.38,H-LC,76.04,107352.0,3040.0,14493.0,-0.61,2.91,13.50,16.81,27.0,0.21,0.79,19.43,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,33.33,128358.0,-14121.0,14748.0,-0.31,-9.91,11.49,0.44,247.0,-0.96,0.94,51.50,OX40N,NTT,BANKS
4,APOLLOHOSP,6904.43,8285.00,-20.84,H-LC,81.77,107842.0,4276.0,16435.0,-2.30,4.13,15.24,20.00,32.0,0.26,0.79,18.78,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,NATIONALUM,189.63,247.44,-50.15,H-MC,58.33,100435.0,-69.0,30713.0,0.29,-0.07,30.58,30.49,97.0,-0.0,0.74,32.31,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,HINDUNILVR,2413.81,2723.0,-9.90,X-LC,89.58,282195.0,11848.0,22773.0,4.62,4.38,8.07,12.81,5.0,0.52,2.07,18.02,XY25,NTT,FMCG
56,LAOPALA,369.40,464.0,126.94,H-SC,41.67,72154.0,-28692.0,54520.0,3.67,-28.45,75.56,25.61,136.0,-0.53,0.53,30.13,AR,NTT,CERAMICS
94,WHIRLPOOL,1167.49,2270.0,-38.81,M-SC,72.92,201758.0,30137.0,131930.0,3.33,17.56,65.39,94.43,201.0,0.23,1.48,48.91,XR,NTT,DURABLES
52,JPPOWER,18.73,26.2,-10.80,L-SC,100.00,178950.0,37108.0,19470.0,2.61,26.16,10.88,39.88,262.0,1.91,1.32,81.35,XY24,NTT,POWER
92,VIPIND,488.80,489.0,-223.38,H-SC,94.79,89027.0,-6289.0,6330.0,1.73,-6.60,7.11,0.04,157.0,-0.99,0.65,75.60,OX40N,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,36.46,18065.0,-31528.0,847.0,-4.48,-63.57,4.69,-61.86,270.0,-37.22,0.13,21.15,XY24,ATH,MISC
69,SAMMAANCAP,170.35,326.00,-165.91,M-SC,38.54,76326.0,-25884.0,119275.0,-3.63,-25.32,156.27,91.37,212.0,-0.22,0.56,23.90,XY25,NTT,FINANCE
65,RELAXO,902.64,1176.00,-29.12,H-SC,97.92,64102.0,-51436.0,86422.0,-3.48,-44.52,134.82,30.28,131.0,-0.60,0.47,25.23,X40N,NTT,FOOTWEAR
83,TCS,3794.03,4998.00,-21.92,X-LC,15.62,254748.0,-41186.0,135093.0,-3.43,-13.92,53.03,31.73,1.0,-0.30,1.87,1.97,X40,BTT,IT
3,ALKYLAMINE,2347.98,4546.37,3.23,X-SC,96.88,96071.0,-4892.0,99424.0,-3.27,-4.85,103.49,93.63,67.0,-0.05,0.71,43.10,SR,ATH,CHEMICALS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-223.38,H-SC,94.79,89027.0,-6289.0,6330.0,1.73,-6.60,7.11,0.04,157.0,-0.99,0.65,75.60,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,33.33,128358.0,-14121.0,14748.0,-0.31,-9.91,11.49,0.44,247.0,-0.96,0.94,51.50,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,31.25,150558.0,-20583.0,20521.0,-0.93,-12.03,13.63,-0.03,191.0,-1.00,1.11,8.21,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,87.50,148825.0,-25734.0,25762.0,0.00,-14.74,17.31,0.02,133.0,-1.00,1.09,29.05,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-19.16,X-MC,5.21,84924.0,-17728.0,17724.0,-1.16,-17.27,20.87,-0.00,59.0,-1.00,0.62,29.09,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-21.92,X-LC,15.62,254748.0,-41186.0,135093.0,-3.43,-13.92,53.03,31.73,1.0,-0.30,1.87,1.97,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.98,X-LC,39.58,285487.0,23886.0,121732.0,-1.29,9.13,42.64,55.67,2.0,0.20,2.10,15.79,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-9.90,X-LC,89.58,282195.0,11848.0,22773.0,4.62,4.38,8.07,12.81,5.0,0.52,2.07,18.02,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,15.59,X-LC,75.00,264891.0,35658.0,31628.0,-2.03,15.56,11.94,29.35,10.0,1.13,1.95,25.86,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-7.83,X-LC,10.42,77942.0,-32230.0,121301.0,-2.32,-29.25,155.63,80.85,11.0,-0.27,0.57,14.80,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.7,NaN,NaN,98.96,79418.0,44969.0,0.0,1.16,130.54,0.00,130.54,274.0,inf,0.58,21.44,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-10.8,L-SC,100.00,178950.0,37108.0,19470.0,2.61,26.16,10.88,39.88,262.0,1.91,1.32,81.35,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.9,X-SC,91.67,251537.0,46060.0,35265.0,0.12,22.42,14.02,39.58,65.0,1.31,1.85,34.79,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,66.67,231316.0,6354.0,161297.0,-0.33,2.82,69.73,74.53,189.0,0.04,1.70,34.30,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,28.30,H-SC,48.96,131355.0,6058.0,53146.0,-2.42,4.83,40.46,47.25,143.0,0.11,0.97,64.33,MH,ATH,POWER
20,CAMS,3643.00,5250.99,1.42,H-SC,26.04,113949.0,11945.0,33079.0,-2.82,11.71,29.03,44.14,126.0,0.36,0.84,29.85,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.05,H-SC,93.75,179980.0,15310.0,53778.0,0.17,9.30,29.88,41.95,124.0,0.28,1.32,17.46,X40N,ATH,AC
88,UNIONBANK,123.87,163.00,-6.01,M-LC,14.58,163432.0,22592.0,21900.0,-0.08,16.04,13.40,31.59,88.0,1.03,1.20,41.97,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,98.96,79418.0,44969.0,0.0,1.16,130.54,0.00,130.54,274.0,inf,0.58,21.44,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.81,M-SC,72.92,201758.0,30137.0,131930.0,3.33,17.56,65.39,94.43,201.0,0.23,1.48,48.91,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-46.96,H-SC,84.38,138044.0,14708.0,122252.0,0.71,11.93,88.56,111.05,123.0,0.12,1.01,35.26,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,28.30,H-SC,48.96,131355.0,6058.0,53146.0,-2.42,4.83,40.46,47.25,143.0,0.11,0.97,64.33,MH,ATH,POWER
95,WIPRO,243.46,420.00,-6.49,M-LC,59.38,160146.0,9201.0,100251.0,-2.55,6.10,62.60,72.51,71.0,0.09,1.18,10.31,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,36.46,18065.0,-31528.0,847.0,-4.48,-63.57,4.69,-61.86,270.0,-37.22,0.13,21.15,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,11.46,66797.0,-46752.0,86803.0,-0.26,-41.17,129.95,35.27,265.0,-0.54,0.49,63.49,XR,NTT,HOTELS
32,ENRIN,1377.95,3176.70,NaN,NaN,98.96,79418.0,44969.0,0.0,1.16,130.54,0.00,130.54,274.0,inf,0.58,21.44,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7211.11,M-SC,81.77,80213.0,-13597.0,90216.0,0.08,-14.49,112.47,81.67,245.0,-0.15,0.59,55.34,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-223.38,H-SC,94.79,89027.0,-6289.0,6330.0,1.73,-6.60,7.11,0.04,157.0,-0.99,0.65,75.60,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-10.80,L-SC,100.00,178950.0,37108.0,19470.0,2.61,26.16,10.88,39.88,262.0,1.91,1.32,81.35,XY24,NTT,POWER
32,ENRIN,1377.95,3176.70,NaN,NaN,98.96,79418.0,44969.0,0.0,1.16,130.54,0.00,130.54,274.0,inf,0.58,21.44,AR,ATH,ELECTRICAL
65,RELAXO,902.64,1176.00,-29.12,H-SC,97.92,64102.0,-51436.0,86422.0,-3.48,-44.52,134.82,30.28,131.0,-0.60,0.47,25.23,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,3.23,X-SC,96.88,96071.0,-4892.0,99424.0,-3.27,-4.85,103.49,93.63,67.0,-0.05,0.71,43.10,SR,ATH,CHEMICALS
5,ASIANPAINT,2961.56,4250.00,-16.50,H-LC,95.83,165866.0,-35520.0,123139.0,-0.49,-17.64,74.24,43.51,31.0,-0.29,1.22,15.46,X40,BTT,PAINTS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.97
1,25,41.97
2,50,70.67


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.76
LC    33.57
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.19
X40      12.80
XY25     12.14
X40N     11.86
XR        9.97
OX40N     8.19
AR        8.16
MH        1.71
X5K       1.41
X200      1.38
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.52
M-SC    15.96
H-LC    13.93
X-LC    12.82
H-MC     9.60
X-MC     7.59
M-LC     5.78
X-SC     4.57
M-MC     4.06
L-SC     2.71
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.31,-0.43,34.07
IT,11.88,-12.42,67.63
FINANCE,8.51,-12.79,53.71
BANKS,7.67,-9.42,54.06
MISC,5.88,-19.84,76.05
PAINTS,5.64,-13.64,36.03
ELECTRICAL,5.44,0.59,46.52
HEALTHCARE,4.75,0.34,28.78
AUTO,4.27,-16.60,67.86


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2649062.0,26
XR,1102876.0,13
AR,1041894.0,10
X40,713650.0,10
X40N,549108.0,11
XY25,468584.0,8
OX40N,314222.0,11
SR,182577.0,2
MH,83859.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1992844.0,20
M-SC,1773892.0,21
H-LC,644021.0,12
X-LC,565636.0,8
H-MC,529495.0,8
X-MC,419077.0,6
M-MC,417729.0,3
X-SC,299157.0,5
M-LC,275642.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      899587.0      6
M-SC       XY24      806047.0      7
H-SC       AR        524166.0      3
M-SC       XR        336933.0      4
M-MC       XY24      301706.0      2
H-MC       XY24      298260.0      3
X-LC       X40       293120.0      3
H-SC       XR        221694.0      3
           X40N      207289.0      4
H-LC       X40N      206768.0      4
M-SC       AR        188922.0      2
           OX40N     168434.0      5
H-LC       X40       161066.0      3
M-LC       XY24      160991.0      2
X-MC       X40       159114.0      2
L-SC       XR        150965.0      2
X-SC       XR        123366.0      1
X-LC       AR        121301.0      1
M-SC       XY25      119275.0      1
M-MC       XR        116023.0      1
X-MC       XY25      108496.0      1
M-LC       XR        100251.0      1
X-SC       SR         99424.0      1
X-LC       XY25       95514.0      3
H-SC       OX40N      86962.0      3
H-LC       AR         84930.0      1
M-SC       SR         83153.0      1
H-MC       XY25       82355.0      1
X-MC       XY24       79625.0      1
H-LC       X5K        75560.0      2
           X200       72867.0      1
M-SC       X40        71128.0      1
H-MC       AR         63713.0      1
L-SC       AR         58862.0      1
X-LC       X40N       55701.0      1
X-MC       X40N       54118.0      1
L-MC       XR         53644.0      1
H-SC       MH         53146.0      1
L-LC       XY25       48544.0      1
H-LC       XY24       42830.0      1
X-SC       OX40N      41102.0      2
           XY24       35265.0      1
H-MC       MH         30713.0      1
           X40        29222.0      1
           X40N       25232.0      1
L-SC       XY24       19470.0      1
X-MC       OX40N      17724.0      1
M-LC       XY25       14400.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
